
<div style="background-color:white; text-align:center; padding:20px;">
    <h2 style="color:black; font-family: Verdana, sans-serif;"><strong>Warlord</strong></h2>
    <p style="font-size: 14px; color: black; font-family: Verdana, sans-serif;">
        <table style="margin: auto; border-collapse: collapse;">
            <tr>
                <th style="border: 0;">Names</th>
                <th style="border: 0;">GitHub Username</th>
            </tr>
            <tr>
                <td style="border: 0;">Vince Ammerlaan</td>
                <td style="border: 0;">Vince16270</td>
            </tr>
            <tr>
                <td style="border: 0;">Tommi Lander</td>
                <td style="border: 0;">tommi7</td>
            </tr>
            <tr>
                <td style="border: 0;">Nima Ghafar</td>
                <td style="border: 0;">NimaGhafar</td>
            </tr>
            <tr>
                <td style="border: 0;">Isa Dijkstra</td>
                <td style="border: 0;">IsaD01</td>
            </tr>
        </table>
    </p>
</div>

<div style="display: flex; justify-content: center; align-items: center; margin-top: 10px;">
    <img src="photo/image1.png" alt="" style="width: 1000px; height: auto;">
</div>


- Docent: Vikram Radhakrishnan
- Datum: 22-06-2025

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Inhoudsopgave </strong></h2>
</div>
<ul style="padding: 0; list-style: none;">
    <li style="text-align: center;"><a href="#1.0">Chapter 1: Het project</a></li>
    <li style="text-align: center;"><a href="#2.0">Chapter 2: Packages en functies inladen</a></li>
    <li style="text-align: center;"><a href="#3.0">Chapter 3: Trainen van de agents</a></li>
    <li style="text-align: center;"><a href="#4.0">Chapter 4: Resultaten van het trainen</a></li>
    <li style="text-align: center;"><a href="#5.0">Chapter 5: Conclusie</a></li>
    <li style="text-align: center;"><a href="#5.0">Chapter 6: Bronnenlijst</a></li>
</ul>

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 1: Het project </strong></h2>

</ul>

In deze opdracht richten we ons op het computerspel Warlords. In dit spel beschermd elke speler een kasteel dat zich in een van de vier hoeken van het scherm bevindt. Vanuit het midden van het scherm komt een bal die wordt teruggekaatst om de kastelen van de tegenstanders te vernietigen. Elke speler heeft een schild waarmee hij de bal kan afweren of richten op vijanden.

Ons doel is om een slimme agent te ontwikkelen die zelfstandig leert spelen via multi-agent reinforcement learning. Dat betekent dat de agent leert op basis van zijn eigen ervaringen, maar ook rekening moet houden met het gedrag van andere agents. De omgeving is dynamisch en strategisch: je moet niet alleen goed verdedigen, maar ook slim aanvallen.

Warlords is het meest geschikt voor dit soort leerstrategieën, omdat het meerdere spelers bevat die tegelijk handelen en op elkaar reageren. Dat maakt het leerproces complex en interessant: agents moeten niet alleen leren van hun eigen fouten, maar ook anticiperen op anderen.

Het spel eindigt op twee manieren:

- Je wint door als laatste overgebleven speler in het spel te zijn.
- Je verliest als je kasteel volledig vernietigd is door de bal.

**Wat is het probleem?**

We willen een slimme agent ontwikkelen die zelfstandig leert om het spel Warlords goed te spelen. Deze agent moet:

- Leren hoe de bal beweegt en hoe hij daarop moet reageren
- Ontwikkelen hoe je effectief verdedigt én aanvalt
- Tactisch aanvallen uitvoeren om de vijandelijke kastelen te verzwakken
- Opletten op meerdere tegenstanders tegelijk en zich aanpassen aan hun gedrag

**Wat is het doel?**

De agent moet:

- Zelf leren van hun ervaringen en hun gedrag continu aanpassen
- Beter presteren dan agents die willekeurig bewegen of vaste strategieën gebruiken
- Verschillende strategieën ontwikkelen afhankelijk van tegenstanders
- Leren overleven in een complexe omgeving en uiteindelijk winnen


---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 2: Packages en functies inladen </strong></h2>

</ul>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install the necessary libraries
!pip install pettingzoo[atari]
!pip install "autorom[accept-rom-license]"
!pip install --find-links dist/ --no-cache-dir AutoROM[accept-rom-license]
!pip install supersuit

Looking in links: dist/


In [ ]:
# Start AutoROM

!AutoROM

# Import libraries
import torch
import supersuit as ss
from pettingzoo.atari import warlords_v3
from pettingzoo.utils import BaseParallelWrapper
import gymnasium as gym
import numpy as np
from collections import defaultdict, Counter
import importlib
import os
import imageio

AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.11/dist-packages/AutoROM/roms
	/usr/local/lib/python3.11/dist-packages/multi_agent_ale_py/roms

Existing ROMs will be overwritten.

I own a license to these Atari 2600 ROMs.
I agree to not distribute these ROMs and wish to proceed: [Y/n]: Y


---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 3: Trainen van de agents - Hyperparameter experiment </strong></h2>

</ul>

**PPO Agent: Proximal Policy Optimization**

Wij hebben ervoor gekozen om een PPO-agent te trainen. PPO staat voor Proximal Policy Optimization en is een state-of-the-art algoritme dat agents in staat stelt om zelfstandig te leren door middel van trial-and-error in een omgeving.

De drie belangrijkste voordelen van PPO zijn:

- Eenvoud
- Stabiliteit
- Steekproefefficiëntie


Het PPO-algoritme werkt volgens de volgende principes:

- *Policy Gradient Methods*:
PPO leert direct welk gedrag (actie) optimaal is in een bepaalde situatie (state), in plaats van alleen in te schatten hoe goed een situatie op zichzelf is. In onze agent is dit terug te zien in hoe acties worden gekozen: op basis van een kansverdeling die voortkomt uit een getraind neuraal netwerk.
- *Objective Function*:
Het algoritme probeert het gedrag van de agent zo aan te passen dat deze op de lange termijn zoveel mogelijk beloning ontvangt uit de omgeving. Dit gebeurt aan de hand van een speciaal ontworpen loss functie die gebaseerd is op de verwachte toekomstige beloningen. Onze agent optimaliseert deze functie om het gedrag voortdurend te verbeteren.

- *Clipped Surrogate Objective*:
PPO voorkomt dat de agent na één leerstap drastisch ander gedrag gaat vertonen. Grote aanpassingen in het gedrag worden ‘geclipt’, waardoor het leerproces stabiel blijft. Dit principe is verwerkt in de manier waarop wij de verhouding tussen oude en nieuwe actiekeuzes beperken tijdens het optimaliseren.

- *Multiple Epochs & Mini-Batch Updates*:
De agent verzamelt ervaringen (observaties, acties, beloningen) in de omgeving. Deze gegevens worden vervolgens herhaaldelijk gebruikt tijdens het leerproces, opgesplitst in kleine, efficiënte mini-batches.

- *Value Function Estimation*:
PPO maakt ook gebruik van een waarde-inschatting van situaties (de ‘value function’), wat helpt om het leerproces minder schommelend en efficiënter te maken. In onze agent is dit geïntegreerd in het neurale netwerk dat zowel de actievoorspellingen als de waarde-inschattingen doet. Deze waarde-inschatting maakt het leren sneller en stabieler.

- *Parallelization*:
Het algoritme is geschikt voor parallel leren (bijvoorbeeld op meerdere CPU’s of GPU’s), wat de trainingssnelheid aanzienlijk verhoogt. Onze agent is zodanig opgezet dat deze op een GPU kan draaien.

We maken ook gebruik van geheugen. Onze agent houdt tijdens interactie met de omgeving continu bij welke acties, beloningen en observaties hij tegenkomt. Deze worden tijdelijk opgeslagen in een geheugenstructuur en vervolgens gebruikt tijdens het leerproces

(DhanushKumar, 2024)

In [44]:
import sys
import os, torch, supersuit as ss, imageio
from pettingzoo.atari import warlords_v3
from random_agent import Agent1
import train_ppo
from IPython.display import Video, display

os.makedirs("Hyperparam", exist_ok=True)

%cd /content/drive/MyDrive/Warlord/src

sys.path.append(os.getcwd())
!ls

env = warlords_v3.env()
env.reset()
print(env.agent_selection)

/content/drive/MyDrive/Warlord/src
env.py	       Hyperparam   ppo_agent.py  random_agent.py  train.py
google_collab  __init__.py  __pycache__   train_ppo.py	   utils.py
first_0


In [45]:
def make_wrapped_env(render=False):
    env = warlords_v3.env(
        max_cycles = 10_000,
        render_mode = "rgb_array" if render else None
    )
    env = ss.max_observation_v0(env, 2)
    env = ss.frame_skip_v0(env, 4)
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, stack_size=4)
    env = ss.dtype_v0(env, dtype=float)
    env = ss.normalize_obs_v0(env, env_min=0, env_max=255)
    return env

def get_action(agent_obj, obs):
    if hasattr(agent_obj, "predict"):
        a, _ = agent_obj.predict(obs, deterministic=True)
    elif hasattr(agent_obj, "choose_action"):
        a, _ = agent_obj.choose_action(obs)
    else:
        a = agent_obj.act(obs)
    return a

**Training van 4 PPO agents met andere hyperparameters**

In [46]:
experiments = [
    {"name":"exp1", "lr":3e-4, "gamma":0.99, "eps_clip":0.20},
    {"name":"exp2", "lr":5e-5, "gamma":0.99, "eps_clip":0.20},
    {"name":"exp3", "lr":3e-4, "gamma":0.99, "eps_clip":0.30},
    {"name":"exp4", "lr":3e-4, "gamma":0.90, "eps_clip":0.20},
]

TOTAL_STEPS = 15_000
UPDATE_INT  = 2_048
trained     = {}

for exp in experiments:
    print(f"\nTraining {exp['name']} ({TOTAL_STEPS} steps)")
    agents = train_ppo.train_warlords_ppo(
        total_timesteps = TOTAL_STEPS,
        update_interval = UPDATE_INT,
        lr       = exp["lr"],
        gamma    = exp["gamma"],
        eps_clip = exp["eps_clip"])

    trained[exp["name"]] = agents
    main_id = list(agents.keys())[0]
    torch.save(agents[main_id].policy.state_dict(),
               f"Hyperparam/{exp['name']}.pth")


Training exp1 (15000 steps)
Using device: cuda
Starting training...
Timestep: 363/15000, Episode 1 finished.
Timestep: 517/15000, Episode 2 finished.
Timestep: 1426/15000, Episode 3 finished.
Timestep: 1748/15000, Episode 4 finished.
Timestep: 2602/15000, Episode 5 finished.
Timestep: 3455/15000, Episode 6 finished.
Timestep: 4381/15000, Episode 7 finished.
Timestep: 5084/15000, Episode 8 finished.
Timestep: 5749/15000, Episode 9 finished.
Timestep: 6415/15000, Episode 10 finished.
Timestep: 7092/15000, Episode 11 finished.
Timestep: 8078/15000, Episode 12 finished.
Timestep: 8828/15000, Episode 13 finished.
Timestep: 9267/15000, Episode 14 finished.
Timestep: 10675/15000, Episode 15 finished.
Timestep: 11495/15000, Episode 16 finished.
Timestep: 11955/15000, Episode 17 finished.
Timestep: 12066/15000, Episode 18 finished.
Timestep: 12501/15000, Episode 19 finished.
Timestep: 13233/15000, Episode 20 finished.
Timestep: 14204/15000, Episode 21 finished.
Timestep: 14504/15000, Episode 2

**Laat de vier PPO agents tegen elkaar spelen**

In [47]:
slot_names = make_wrapped_env().possible_agents
policy_map = {}
for idx, exp in enumerate(experiments):
    pol_id = list(trained[exp["name"]].keys())[0]
    policy_map[ slot_names[idx] ] = trained[exp["name"]][pol_id]

def play_four_way(agent_map, episodes=10):
    env = make_wrapped_env()
    totals = {a: 0.0 for a in env.possible_agents}
    for _ in range(episodes):
        env.reset()
        done = {a: False for a in env.agents}
        while not all(done.values()):
            for agent in env.agent_iter():
                obs, rew, term, trunc, _ = env.last()
                if term or trunc:
                    env.step(None); done[agent] = True; continue
                env.step(get_action(agent_map[agent], obs))
                totals[agent] += rew
    env.close()
    return {k: v/episodes for k,v in totals.items()}

avg = play_four_way(policy_map, episodes=1)

print("\nGemiddelde reward per agent (10 potjes):")
for slot in slot_names:
    print(f"{slot}: {avg[slot]:.2f}")

best_slot = max(avg, key=avg.get)
torch.save(policy_map[best_slot].policy.state_dict(), "Hyperparam/best_agent.pth")
print(f"\nBeste policy = {best_slot}  (reward {avg[best_slot]:.2f}) opgeslagen: Hyperparam/best_agent.pth")


Gemiddelde reward per agent (10 potjes):
first_0: 0.00
second_0: 0.00
third_0: 0.00
fourth_0: 0.00

Beste policy = first_0  (reward 0.00) opgeslagen: Hyperparam/best_agent.pth


## **Chapter 3.1: Agent tegen Baseline**

In [56]:
import os, torch, supersuit as ss
from pettingzoo.atari import warlords_v3
from ppo_agent import PPOAgent
from random_agent import Agent1

In [57]:
def make_wrapped_env(render=False):
    env = warlords_v3.env(
        max_cycles = 10_000,
        render_mode = "rgb_array" if render else None)

    env = ss.max_observation_v0(env, 2)
    env = ss.frame_skip_v0(env, 4)
    env = ss.resize_v1(env, x_size=84, y_size=84)
    env = ss.frame_stack_v1(env, stack_size=4)
    env = ss.dtype_v0(env, dtype=float)
    env = ss.normalize_obs_v0(env, env_min=0, env_max=255)
    return env

def get_action(agent_obj, obs):
    if hasattr(agent_obj, "predict"):
        a, _ = agent_obj.predict(obs, deterministic=True)
    elif hasattr(agent_obj, "choose_action"):
        a, _ = agent_obj.choose_action(obs)
    else:
        a = agent_obj.act(obs)
    return a

**Laad het beste PPO-model**

In [58]:
ckpt = "Hyperparam/best_agent.pth"
assert os.path.exists(ckpt), "Train eerst en zorg dat best_agent.pth er is"

tmp_env   = make_wrapped_env()
obs_shape = tmp_env.observation_space("first_0").shape
action_n  = tmp_env.action_space("first_0").n
tmp_env.close()

best_policy = PPOAgent(obs_shape, action_n, lr=1e-4)
best_policy.policy.load_state_dict(torch.load(ckpt, map_location="cpu"))
best_policy.policy.eval()
print("Beste PPO-policy geladen:", ckpt)

Beste PPO-policy geladen: Hyperparam/best_agent.pth


**Laat het PPO model spelen tegen de random-agent.**

In [59]:
def evaluate_vs_random(policy, episodes=10):
    env     = make_wrapped_env()
    totals  = {a: 0.0 for a in env.possible_agents}

    for _ in range(episodes):
        env.reset()
        done = {a: False for a in env.agents}
        while not all(done.values()):
            for agent in env.agent_iter():
                obs, rew, term, trunc, _ = env.last()
                if term or trunc:
                    env.step(None); done[agent] = True; continue

                act = get_action(policy if agent=="first_0" else Agent1(), obs)
                env.step(act)
                totals[agent] += rew
    env.close()
    return {k: v/episodes for k, v in totals.items()}

avg = evaluate_vs_random(best_policy, episodes=10)

print("\nGemiddelde reward vs 3× random (10 potjes):")
for agent_name, reward in avg.items():
    print(f"{agent_name:8}  {reward:.2f}")


Gemiddelde reward vs 3× random (10 potjes):
first_0   0.00
second_0  0.00
third_0   0.00
fourth_0  0.00


---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 4: Resultaten van het trainen </strong></h2>

</ul>

**Omdat we geen echte rewards hebben gekregen hebben we ook geen rewards.csv, hierdoor geen visualisaties**

In [54]:
import os, glob, torch, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from pettingzoo.atari import warlords_v3
import supersuit as ss
from random_agent import Agent1
from ppo_agent import PPOAgent

In [55]:
import pandas as pd, matplotlib.pyplot as plt, numpy as np, glob, os
from pathlib import Path

log_files = glob.glob("Hyperparam/exp*_rewards.csv")
assert log_files, "Geen rewards.csv bestand gevonden."

plt.style.use("seaborn-v0_8")

AssertionError: Geen rewards.csv bestand gevonden.

**Visualiseer de reward curves**


In [ ]:
fig, ax = plt.subplots(figsize=(9,4))
for file in sorted(log_files):
    tag = Path(file).stem.split("_")[0]           # 'exp1' etc.
    df  = pd.read_csv(file, names=["episode","reward","timestep"])
    df["smooth"] = df.reward.rolling(50).mean()

    ax.plot(df.episode, df.reward, alpha=.25, label=f"{tag} raw")
    ax.plot(df.episode, df.smooth, lw=2, label=f"{tag} smooth")

ax.set_xlabel("Episode"); ax.set_ylabel("Reward")
ax.set_title("Reward-curves per experiment")
ax.legend(ncol=2); plt.show()

**Rolling STD**

In [ ]:
fig, ax = plt.subplots(figsize=(9,3))
for file in sorted(log_files):
    tag = Path(file).stem.split("_")[0]
    df  = pd.read_csv(file, names=["episode","reward","timestep"])
    ax.plot(df.episode, df.reward.rolling(50).std(), label=tag)
ax.set_xlabel("Episode"); ax.set_ylabel("Rolling σ (50)")
ax.set_title("Training-stabiliteit"); ax.legend(); plt.show()

**Histogram van laatste 100 episodes**

In [ ]:
LAST = 100
plt.figure(figsize=(6,3))
for file in sorted(log_files):
    tag = Path(file).stem.split("_")[0]
    df  = pd.read_csv(file, names=["episode","reward","timestep"])
    df.tail(LAST).reward.plot.hist(alpha=.4, bins=25, label=tag)
plt.title(f"Verdeling rewards (laatste {LAST} episodes)")
plt.xlabel("Reward"); plt.legend(); plt.show()

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 5: Conclusie </strong></h2>
</div>

Ondanks meerdere experimenten wat varieerde van 2 000 tot 200 000 trainings­stappen, verschillende learning-rates, γ-waarden en clipranges bleven alle agents zonder beloning. We hebben ook:

Potjes verlengd naar 10 000 cycli om ervoor te zorgen dat er blokken gebroken konden worden en Frame-skip-waarden aangepast om het aantal interacties te vergroten.

Toch leverden geen van deze instellingen positieve rewards op, waardoor we de prestaties van het PPO-model tegenover de baseline­random agent niet konden beoordelen.

---
<div style="background-color:white; text-align:center; vertical-align:middle; padding:50px 0; margin-top:5px; margin-bottom:5px">
    <h2 id="eda-title" style="color:black; font-family: Verdana, sans-serif; font-size: 25px;"><strong> Chapter 6: Bronnenlijst </strong></h2>
</div>

- DhanushKumar. (2024, 12 mei). PPO algorithm. Medium. https://medium.com/@danushidk507/ppo-algorithm-3b33195de14a

- Schulman, J., Wolski, F., Dhariwal, P., Radford, A., & Klimov, O. (2017). Proximal policy optimization algorithms (arXiv:1707.06347). arXiv. https://arxiv.org/abs/1707.06347

- Weng, L. (2018, 8 april). Policy gradient algorithms. Lil’Log. https://lilianweng.github.io/posts/2018-04-08-policy-gradient/

- OpenAI. (z.d.). Proximal policy optimization (PPO). Spinning Up in Deep RL. https://spinningup.openai.com/en/latest/algorithms/ppo.html